<a href="https://colab.research.google.com/github/facklen/python-chatbot-challenge/blob/main/notebooks/Dia2_Configuracao_Ngrok.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Configuração do Ngrok - Dia 2 do #7DaysOfCode

Neste notebook, configurei o Ngrok no Google Colab para permitir acesso externo às aplicações desenvolvidas durante o desafio.

## O que é o Ngrok?
O Ngrok é uma ferramenta que cria túneis seguros para expor serviços locais na internet através de URLs públicas.

## Passos realizados:
1. Verificação da instalação do PyNgrok
2. Configuração do token de autenticação do Ngrok
3. Teste de funcionalidade criando um túnel temporário
4. Criação de uma função reutilizável para os próximos dias do desafio

## Observações:
- É necessário reconfigurar o token de autenticação em cada nova sessão do Colab
- O plano gratuito do Ngrok tem algumas limitações, como número de conexões simultâneas

# Verificação da instalação do PyNgrok

In [1]:
# Verificar se o pyngrok já está instalado
try:
    from pyngrok import ngrok
    print("PyNgrok já está instalado!")
except ImportError:
    print("PyNgrok não está instalado. Instalando agora...")
    !pip install pyngrok -q
    print("PyNgrok instalado com sucesso!")

PyNgrok não está instalado. Instalando agora...
PyNgrok instalado com sucesso!


# Criação de conta no Ngrok

In [2]:
# Execute esta célula e clique no link para criar uma conta no Ngrok (ignore se já tiver)
from IPython.display import display, HTML
display(HTML('<a href="https://dashboard.ngrok.com/signup" target="_blank">Clique aqui para criar uma conta no Ngrok</a>'))

# Configuração do token de autenticação do Ngrok

In [ ]:
# Obtenha seu authtoken em: https://dashboard.ngrok.com/auth/your-authtoken
import os
from IPython.display import display, HTML

display(HTML('<a href="https://dashboard.ngrok.com/auth/your-authtoken" target="_blank">Clique aqui para obter seu Ngrok authtoken</a>'))

# Função para configurar o token do Ngrok
def configurar_ngrok_token():
    token = input("Cole aqui seu token do Ngrok: ")
    !ngrok authtoken {token}
    # Também salvar o token para uso no código Python
    os.environ["NGROK_AUTH_TOKEN"] = token
    print("Token do Ngrok configurado com sucesso!")

configurar_ngrok_token()

# Teste do funcionamento do Ngrok

In [6]:
# Teste de funcionamento do Ngrok
from pyngrok import ngrok
import time

# Listar túneis ativos (deve estar vazio inicialmente)
print("Túneis ativos antes de criar um novo:")
print(ngrok.get_tunnels())

# Criar um túnel de teste (na porta 8000)
print("\nCriando um túnel de teste...")
http_tunnel = ngrok.connect(8000)
print(f"URL pública do Ngrok: {http_tunnel.public_url}")

# Listar túneis ativos novamente
print("\nTúneis ativos depois de criar um novo:")
print(ngrok.get_tunnels())


Túneis ativos antes de criar um novo:
[]

Criando um túnel de teste...
URL pública do Ngrok: https://d7b8-34-106-183-154.ngrok-free.app

Túneis ativos depois de criar um novo:
[<NgrokTunnel: "https://d7b8-34-106-183-154.ngrok-free.app" -> "http://localhost:8000">]


# Encerrando o túnel

In [7]:
# Esperar 5 segundos para que você possa ver o resultado
time.sleep(5)

# Encerrar o túnel
print("\nEncerrando o túnel...")
ngrok.disconnect(http_tunnel.public_url)
ngrok.kill()
print("Túnel encerrado!")


Encerrando o túnel...
Túnel encerrado!


# Criação de servidor HTTP Básico


*   Função iniciar_servidor_teste - inicializa um servidor de teste
*   Função finalizar_servidor - finaliza servidor de teste



In [14]:
def iniciar_servidor_teste(porta=8000):
    """
    Inicia um servidor HTTP simples para testar o túnel Ngrok.

    Args:
        porta (int): Porta para iniciar o servidor. Padrão: 8000
    """
    from http.server import HTTPServer, BaseHTTPRequestHandler
    import threading

    # Definir um manipulador HTTP simples
    class MeuManipulador(BaseHTTPRequestHandler):
        def do_GET(self):
            self.send_response(200)
            self.send_header('Content-type', 'text/html')
            self.end_headers()

            # Página HTML simples
            html = f"""
            <html>
              <head>
                <title>Teste do Chatbot - #7DaysOfCode</title>
                <style>
                  body {{ font-family: Arial, sans-serif; max-width: 800px; margin: 0 auto; padding: 20px; }}
                  h1 {{ color: #4285f4; }}
                  .container {{ border: 1px solid #ddd; padding: 20px; border-radius: 5px; }}
                </style>
              </head>
              <body>
                <h1>Servidor de Teste para o #7DaysOfCode</h1>
                <div class="container">
                  <h2>Chatbot em Python</h2>
                  <p>Se você está vendo esta página, significa que:</p>
                  <ul>
                    <li>O servidor HTTP está funcionando corretamente</li>
                    <li>O túnel Ngrok está encaminhando o tráfego adequadamente</li>
                    <li>A autenticação básica está funcionando</li>
                  </ul>
                  <p>Caminho atual: {self.path}</p>
                  <p>Endereço: {self.client_address}</p>
                </div>
              </body>
            </html>
            """

            self.wfile.write(html.encode())

    # Iniciar o servidor em uma thread separada
    server = HTTPServer(('', porta), MeuManipulador)
    print(f"Servidor iniciado na porta {porta}")
    print(f"Acesse em http://localhost:{porta} no ambiente local")
    print("Para encerrar o servidor, você precisará reiniciar o runtime do Colab")

    # Executar o servidor em uma thread separada para não bloquear o notebook
    thread = threading.Thread(target=server.serve_forever)
    thread.daemon = True  # Permite que o notebook seja encerrado mesmo com o servidor rodando
    thread.start()

    return server

def finalizar_servidor(servidor):
    """
    Finaliza um servidor HTTP iniciado pela função iniciar_servidor_teste.

    Args:
        servidor: Instância do servidor HTTP a ser finalizado.
    """
    if servidor:
        print("Finalizando o servidor HTTP...")
        servidor.shutdown()  # Encerra o loop serve_forever()
        servidor.server_close()  # Fecha o socket
        print("Servidor HTTP finalizado com sucesso.")
    else:
        print("Nenhum servidor para finalizar.")

# Criação de uma função reutilizável para o Ngrok


*   Função criar_tunel_ngrok - cria túnel ngrok
*   Função encerrar_tunel_ngrok - finaliza túnel ngrok



In [13]:
# Função reutilizável para gerenciar túneis Ngrok
def criar_tunel_ngrok(porta, auth_token=None,  usuario="admin", senha="chatbot123"):
    """
    Cria um túnel Ngrok para a porta especificada com autenticação básica.

    Args:
        porta (int): Porta local que você deseja expor.
        auth_token (str, opcional): Token de autenticação do Ngrok.
                                   Se não fornecido, solicita ao usuário.
        usuario (str, opcional): Nome de usuário para autenticação básica. Padrão: "admin"
        senha (str, opcional): Senha para autenticação básica. Padrão: "chatbot123"

    Returns:
        str: URL pública do túnel Ngrok.
    """
    from pyngrok import ngrok, conf
    import os

    # Verificar se o token está definido
    if not auth_token:
        if "NGROK_AUTH_TOKEN" not in os.environ:
            auth_token = input("Cole aqui seu token do Ngrok: ")
        else:
            auth_token = os.environ["NGROK_AUTH_TOKEN"]

    # Configurar o token
    os.environ["NGROK_AUTH_TOKEN"] = auth_token
    !ngrok authtoken {auth_token}

    # Configurar autenticação básica
    # O formato é: "usuario:senha"
    auth = f"{usuario}:{senha}"

    # Configurar e iniciar o túnel com autenticação básica
    pyngrok_config = conf.PyngrokConfig()
    public_url = ngrok.connect(
        addr=porta,
        auth=auth,
        pyngrok_config=pyngrok_config
    )

    print(f"Túnel Ngrok criado com sucesso!")
    print(f"URL pública: {public_url}")
    print(f"Credenciais de acesso:")
    print(f"  Usuário: {usuario}")
    print(f"  Senha: {senha}")

    return public_url

def encerrar_tunel_ngrok(url_public):
  # Encerrar o túnel
  print("\nEncerrando o túnel...")
  ngrok.disconnect(http_tunnel.public_url)
  ngrok.kill()
  print("Túnel encerrado!")


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Túnel Ngrok criado com sucesso!
URL pública: NgrokTunnel: "https://ceec-34-106-183-154.ngrok-free.app" -> "http://localhost:8000"
Credenciais de acesso:
  Usuário: facklen
  Senha: teste432


# Chamada da função para criação de túnel

In [18]:
# Inicialização do servidor
servidor = iniciar_servidor_teste(porta=8000)
# Criação do túnel
url = criar_tunel_ngrok(porta=8000,usuario="facklen",senha="teste432")

Servidor iniciado na porta 8000
Acesse em http://localhost:8000 no ambiente local
Para encerrar o servidor, você precisará reiniciar o runtime do Colab
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Túnel Ngrok criado com sucesso!
URL pública: NgrokTunnel: "https://2d95-34-106-183-154.ngrok-free.app" -> "http://localhost:8000"
Credenciais de acesso:
  Usuário: facklen
  Senha: teste432


# Encerramento do túnel

In [21]:
# Finaliza servidor
finalizar_servidor(servidor)
# Encerra túnel
encerrar_tunel_ngrok(url)


Encerrando o túnel...
Túnel encerrado!
